# Music Unmixing using MUSDB
## 1. Introduction to the sigsep ecosystem



### Installation of Requirements (**colab only**)

Please use the conda environment or the Pipfile if you want to run this notebook outside of colab in your local environment.



In [ ]:
# update pandas since we don't support 0.24.x
# restart kernel after this update!
!pip install pandas==1.3.5

In [ ]:
!apt install -y libsndfile1 ffmpeg
!pip install musdb
!pip install museval
!pip install -q scikit_posthocs
!pip install librosa

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
     |████████████████████████████████| 963 kB 9.2 MB/s 
     |████████████████████████████████| 130 kB 12.4 MB/s 
     |████████████████████████████████| 43 kB 1.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


### Generic Imports

In [ ]:
from IPython.display import Audio, display
import urllib.request
import numpy as np
import scipy.stats
import seaborn as sns
import pandas as pd
import scikit_posthocs as sp
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.transforms import BlendedGenericTransform
import scikit_posthocs as sp
from urllib.request import urlopen
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import stempeg
import musdb
import librosa
import librosa.display
from scipy.signal import stft, istft
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os.path
from os import path

Mounted at /content/drive


# MUSDB

Download musdb dataset

In [ ]:
mus = musdb.DB(download=True)

Done!


In [ ]:
print(type(mus))
print(len(mus))

<class 'musdb.DB'>
144


In [ ]:
track = mus[42]
f, t, X = stft(track.audio.T, nperseg=4096, noverlap=3072)
print(f.shape)
print(t.shape)
print(X.shape)

(2049,)
(294,)
(2, 2049, 294)


Delete everything in dataset dir (THINK BEFORE YOU RUN!)

In [ ]:
!rm -rf '/content/drive/MyDrive/APS360 Group Project/APS360 Datasets/'

Delete accomp.wav in datase tdir

In [ ]:
root = '/content/drive/MyDrive/APS360 Group Project/Dataset 1'

for track in mus:
  out_dir_path = os.path.join(root,track.name)
  if path.exists(out_dir_path) == True:
    # delete accomp
    out_wav_path = os.path.join(root,track.name,'accomp.wav')
    os.remove(out_wav_path)

Put wav in dataset dir

In [ ]:
root = '/content/drive/MyDrive/APS360 Group Project/Dataset 1'

for track in mus:
  # create dir
  out_dir_path = os.path.join(root,track.name)
  if path.exists(out_dir_path) == False:
    os.mkdir(out_dir_path)

    # get vocals
    vocals = track.targets['mvocals']
    vocals_audio = vocals.audio
    # save vocals
    out_wav_path = os.path.join(root,track.name,'vocals.wav')
    stempeg.write_audio(path=out_wav_path, data=vocals_audio, sample_rate=vocals.rate)

    # get mixture
    mixture = track.targets['linear_mixture']
    mixture_audio = track.audio
    # save mixture
    out_wav_path = os.path.join(root,track.name,'mixture.wav')
    stempeg.write_audio(path=out_wav_path, data=mixture_audio, sample_rate=mixture.rate)
    '''
    # get accompaniment
    accomp = track.targets['accompaniment']
    accomp_audio = accomp.audio
    # save accomp
    out_wav_path = os.path.join(root,track.name,'accomp.wav')
    stempeg.write_audio(path=out_wav_path, data=accomp_audio, sample_rate=accomp.rate)
    '''

Display the file structure

In [ ]:
!apt-get install tree
!tree '/content/drive/MyDrive/APS360 Group Project/Dataset 1/'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tree is already the newest version (1.7.0-5).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
/content/drive/MyDrive/APS360 Group Project/Dataset 1/
├── A Classic Education - NightOwl
│   ├── accomp.wav
│   ├── mixture.wav
│   └── vocals.wav
├── Actions - Devil's Words
│   ├── accomp.wav
│   ├── mixture.wav
│   └── vocals.wav
├── Actions - One Minute Smile
│   ├── accomp.wav
│   ├── mixture.wav
│   └── vocals.wav
├── Actions - South Of The Water
│   ├── accomp.wav
│   ├── mixture.wav
│   └── vocals.wav
├── Aimee Norwich - Child
│   ├── accomp.wav
│   ├── mixture.wav
│   └── vocals.wav
├── Alexander Ross - Goodbye Bolero
│   ├── accomp.wav
│   ├── mixture.wav
│   └── vocals.wav
├── Alexander Ross - Velvet Curtain
│   ├── accomp.wav
│   ├── mixture.wav
│   └── vocals.wav
├── Al James - Schoolboy Facination
│   ├── accomp.wav
│   ├── mixture.wav
│   └── vocals.wav
├── AM Contra 